In [1]:
import requests
import time
import pandas as pd
from itertools import product
import json

In [ ]:
response = requests.post('https://fbrapi.com/generate_api_key')
api_key = response.json()['api_key']
print("API Key:", api_key)

In [2]:
url = "https://fbrapi.com/player-match-stats"
params = {
    "player_id": "92e7e919",
    "league_id": "9",
    "season_id": "2023"
}
headers = {"X-API-Key": f"{api_key}"}

response = requests.get(url, params=params, headers=headers)
print(response.json())

NameError: name 'api_key' is not defined

In [34]:
# Step 1: Get all countries to find Spain
url = "https://fbrapi.com/countries"
headers = {"X-API-Key": f"{api_key}"}

response = requests.get(url, headers=headers)
countries = response.json()
print(countries)

{'message': 'Internal Server Error'}


In [18]:
countries

{'message': 'Internal Server Error'}

In [15]:
# Explore countries data
countries.get('data', [])[65]

IndexError: list index out of range

In [8]:
country_list = list()
for country in countries['data']:
    if country['country'].lower() in ['spain', 'germany', 'italy', 'england', 'france']:
        country_list.append([country['country'], country['country_code']])

print(country_list)

KeyError: 'data'

In [ ]:
# Step 2: Get Tier = 1st Gender = M Leagues for Spain
url = "https://fbrapi.com/leagues"
headers = {"X-API-Key": f"{api_key}"}
params = {"country_code" : "ESP"}

response = requests.get(url, headers=headers, params=params)
leagues = response.json()
# print(type(leagues))

for league in leagues.get('data', []):
    if league.get('league_type') == "domestic_leagues":
        for local_league in league.get('leagues', []):
            if local_league['tier']=='1st' and local_league['gender']=='M':
                print(f"League ID: {local_league['league_id']}, Competition Name: {local_league['competition_name']}")



League ID: 12, Competition Name: La Liga


In [ ]:
# Get 1st tier and 

url = "https://fbrapi.com/leagues"
headers = {"X-API-Key": f"{api_key}"}
domestic_leagues = list()

for country in country_list:
    params = {"country_code" : country[1]}
    response = requests.get(url, headers=headers, params=params)
    leagues = response.json()
    # print(type(leagues))
    
    for league in leagues.get('data', []):
        if league.get('league_type') == "domestic_leagues":
            for local_league in league.get('leagues', []):
                if local_league['tier']=='1st' and local_league['gender']=='M':
                    domestic_leagues.append([country[1],\
                                            local_league['competition_name'],\
                                            local_league['league_id'],\
                                            local_league['first_season'],\
                                            local_league['last_season']])
    time.sleep(10)

In [ ]:
domestic_leagues_df = pd.DataFrame(domestic_leagues, columns=['country_code', 'competition_name', 'league_id', 'first_season', 'last_season'])
display(domestic_leagues_df)

,country_code,competition_name,league_id,first_season,last_season
0,ENG,Premier League,9,1888-1889,2025-2026
1,FRA,Ligue 1,13,1995-1996,2025-2026
2,GER,Fußball-Bundesliga,20,1988-1989,2025-2026
3,ITA,Serie A,11,1988-1989,2025-2026
4,ESP,La Liga,12,1988-1989,2025-2026


In [ ]:
# Get Team IDs for La Liga 2010-2011 season
url = "https://fbrapi.com/team-season-stats"
headers = {"X-API-Key": f"{api_key}"}

teams_list = list()

for year in range(2020, 2026):
    params = {
        "league_id":12,
        "season_id":f"{year-1}-{year}"
        }
    response = requests.get(url, headers=headers, params=params)
    season = response.json()
    
    for teams in season['data']:
        
        teams_list.append([teams['meta_data']['team_name'],teams['meta_data']['team_id']])
    
    time.sleep(5)

teams_unique = sorted(list(set(tuple(team) for team in teams_list)))
len(teams_unique)

27

In [ ]:
league_ids = [league[2] for league in domestic_leagues]
seasons = [str(i)+"-"+str(i+1) for i in range(2010,2025)]

teams_list_top_5 = list()

for league, season in product(league_ids, seasons):
    print(type(league), type(f"{season}"))

<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class 'str'>
<class 'int'> <class

In [ ]:
# Teams IDs for European Top 5 Leagues from the last 15 seasons

url = "https://fbrapi.com/team-season-stats"
headers = {"X-API-Key": f"{api_key}"}

league_ids = [league[2] for league in domestic_leagues]
seasons = [str(i)+"-"+str(i+1) for i in range(2010,2025)]

teams_list_top_5 = list()  # Reset the list

for league, season in product(league_ids, seasons):
    params = {
        "league_id":league,
        "season_id":f"{season[0:4]}"
    }
    
    response = requests.get(url, headers=headers, params=params)
    season_teams_data = response.json()
    
    # Check if 'data' key exists in the response
    if 'data' in season_teams_data:
        print(f"Confirmation 'data' in league {league}, season {season}.")
        for teams in season_teams_data['data']:
            teams_list_top_5.append([teams['meta_data']['team_name'], teams['meta_data']['team_id']])
    else:
        print(f"Warning: No 'data' key for league {league}, season {season}. Response: {season_teams_data}")
    
    time.sleep(5)

teams_list_top_5_unique = sorted(list(set(tuple(team) for team in teams_list_top_5)))
print(f"Total unique teams: {len(teams_list_top_5_unique)}")
print(teams_list_top_5_unique)


In [ ]:
# Save teams_list_top_5_unique as JSON
with open('/home/edgar25/02807_Comp_Tools_DSc/Project/teams_list_top_5_unique.json', 'w') as f:
    json.dump(teams_list_top_5_unique, f, indent=2)
print("Saved: teams_list_top_5_unique.json")

# Save domestic_leagues as JSON
with open('/home/edgar25/02807_Comp_Tools_DSc/Project/domestic_leagues.json', 'w') as f:
    json.dump(domestic_leagues, f, indent=2)
print("Saved: domestic_leagues.json")

print("\nBoth files saved successfully in JSON format!")
print("These files are human-readable and compatible with any programming language.")


Saved: teams_list_top_5_unique.json
Saved: domestic_leagues.json

Both files saved successfully in JSON format!
These files are human-readable and compatible with any programming language.


In [ ]:
# Read teams_list_top_5_unique.json
with open('/home/edgar25/02807_Comp_Tools_DSc/Project/teams_list_top_5_unique.json', 'r') as f:
    teams_data = json.load(f)

print("=" * 70)
print("TEAMS_LIST_TOP_5_UNIQUE.JSON")
print("=" * 70)
print(f"Total teams: {len(teams_data)}\n")
for idx, team in enumerate(teams_data[:10], 1):  # Show first 10
    print(f"{idx}. {team}")
print(f"\n... and {len(teams_data) - 10} more teams\n")

# Read domestic_leagues.json
with open('/home/edgar25/02807_Comp_Tools_DSc/Project/domestic_leagues.json', 'r') as f:
    leagues_data = json.load(f)

print("=" * 70)
print("DOMESTIC_LEAGUES.JSON")
print("=" * 70)
print(f"Total leagues: {len(leagues_data)}\n")
for idx, league in enumerate(leagues_data, 1):
    print(f"{idx}. Country: {league[0]}, League: {league[1]}, ID: {league[2]}, Seasons: {league[3]}-{league[4]}")


TEAMS_LIST_TOP_5_UNIQUE.JSON
Total teams: 96

1. ['Alavés', '8d6fd021']
2. ['Angers', '69236f98']
3. ['Arsenal', '18bb7c10']
4. ['Aston Villa', '8602292d']
5. ['Atalanta', '922493f3']
6. ['Athletic Club', '2b390eca']
7. ['Atlético Madrid', 'db3b9613']
8. ['Augsburg', '0cdc4311']
9. ['Auxerre', '5ae09109']
10. ['Barcelona', '206d90db']

... and 86 more teams

DOMESTIC_LEAGUES.JSON
Total leagues: 5

1. Country: ENG, League: Premier League, ID: 9, Seasons: 1888-1889-2025-2026
2. Country: FRA, League: Ligue 1, ID: 13, Seasons: 1995-1996-2025-2026
3. Country: GER, League: Fußball-Bundesliga, ID: 20, Seasons: 1988-1989-2025-2026
4. Country: ITA, League: Serie A, ID: 11, Seasons: 1988-1989-2025-2026
5. Country: ESP, League: La Liga, ID: 12, Seasons: 1988-1989-2025-2026


In [64]:
url = "https://fbrapi.com/player-season-stats"
params = {
    "team_id" : '206d90db',
    "league_id" : 12,
    "season_id" : '2022-2023'
    }
headers = {"X-API-Key": f"{api_key}"}
response = requests.get(url, headers=headers, params=params)
season_stats = response.json()


In [5]:
sample_dict = {
    "players":[
        {
            "meta_data":{
                "player_id": "8b04d6c1",
                "player_name": "Pierre Højbjerg",
                "player_country_code": "DEN",
                "age": 24
                },
            "stats": {
                "stats" : {
                    "appearances": 34,
                    "goals": 2
                },
                "shooting": {
                    "shots_total": 45,
                    "shots_on_target": 20
                    },
                "misc": {
                    "yellow_cards": 5,
                    "red_cards": 0
                }

            }
        
        },
    ]
}

In [50]:
# Extract nested dictionaries and include items to dict

def extract_nested_dicts(data, depth=0):
    
    for value in data.values():
        if isinstance(value, dict):
            print("Found nested dictionary:")
            extract_nested_dicts(value, depth + 1)
        else:
            depth += 1 
    return depth

In [51]:
extract_nested_dicts(sample_dict)

1

In [56]:
def extract_nested_dicts(input_dict):
    extracted_data = {}
    
    for key, value in input_dict.items():
        if isinstance(value, dict):
            nested_data = extract_nested_dicts(value)
            extracted_data.update(nested_data)
        else:
            extracted_data = {**extracted_data, **{key: value}}
            
    return extracted_data

In [57]:
flat_player_stats = dict()
for player in sample_dict.get("players",[]):
    #print("Player.keys():", player.keys())
    
    player_dict = extract_nested_dicts(player)
        # print("Category:", player[category])  
        # for subcategory in player[category].keys(): # subcategory is str type
        #     print("Subcategory type:", (player[category][subcategory]))
        #     #print(" Subcategory:", subcategory)
        #     player_dict.update({"league_id": 12})
        #     try:
        #         for stat, value in player[category][subcategory].items():
        #             print(type(player[category][subcategory]))
        #             player_dict.update({stat: value})
        #     except AttributeError:
        #         #print(f" Except {subcategory}: {player[category][subcategory]}")
        #         player_dict.update({subcategory: player[category][subcategory]})
    flat_player_stats.update({player['meta_data']['player_id']: player_dict})
        
            
         
       # print(player.keys())
       
    
        # if category == "meta_data":
        #     player_id_ = player[category]["player_id"]
        #     flat_player_stats.update({player_id_: player[category].items()})
        #    # print(player[category].items())
        # elif category == "stats":
        #     for stats_category in player[category].keys():
        #         for stat, value in player[category][stats_category].items():
        #             pass
                    # print(f"{stats_category} - {stat}: {value}")
print(flat_player_stats)

{'8b04d6c1': {'player_id': '8b04d6c1', 'player_name': 'Pierre Højbjerg', 'player_country_code': 'DEN', 'age': 24, 'appearances': 34, 'goals': 2, 'shots_total': 45, 'shots_on_target': 20, 'yellow_cards': 5, 'red_cards': 0}}


In [63]:
flat_player_stats = dict()
for player in sample_dict.get("players",[]):
    #print("Player.keys():", player.keys())
    
    player_dict = extract_nested_dicts(player)
    
    flat_player_stats.update({player['meta_data']['player_id']: player_dict})
print(flat_player_stats)

with open('/home/edgar25/02807_Comp_Tools_DSc/Project/sample_flat_player_stats.json', 'w') as f:
    json.dump(flat_player_stats, f, indent=2)
print("Saved: sample_flat_player_stats.json")

{'8b04d6c1': {'player_id': '8b04d6c1', 'player_name': 'Pierre Højbjerg', 'player_country_code': 'DEN', 'age': 24, 'appearances': 34, 'goals': 2, 'shots_total': 45, 'shots_on_target': 20, 'yellow_cards': 5, 'red_cards': 0}}
Saved: sample_flat_player_stats.json


In [ ]:
url = "https://fbrapi.com/player-season-stats"
headers = {"X-API-Key": f"{api_key}"}

league_ids = json.load(open('Project/domestic_leagues.json'))
seasons = [str(i)+"-"+str(i+1) for i in range(2010,2025)]
teams_ids = json.load(open('Project/teams_list_top_5_unique.json'))


for season in seasons:
    player_season_stats_flat = dict()    
    for league, team in product(league_ids, teams_ids):
        params = {
            "team_id" : team[1],
            "league_id" : league[2],
            "season_id" : season
            }
        response = requests.get(url, headers=headers, params=params)
        players_season_stats = response.json()
        
        for player in players_season_stats.get('players', []):
            player_dict = extract_nested_dicts(player)
            
            player_season_stats_flat.update({player['meta_data']['player_id']: player_dict})

    with open(f"players_season_stats_{season}.json", 'w') as f:
        json.dump(player_season_stats_flat, f, indent=2)

    time.sleep(6)


dict_keys(['player_id', 'player_name', 'player_country_code', 'age']) 
 dict_keys(['stats', 'shooting', 'passing', 'passing_types', 'gca', 'defense', 'possession', 'playingtime', 'misc'])
